In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

# Метод Гаусса

In [2]:
def Gauss(A, b): # A - матрица (n,n), b - вектор (n,1)
    n=len(b)
    #прямой ход метода Гаусса
    for k in range(n): # строки
        # поиск максимума по столбцу
        max_index=0 # индекс максимума к-го столбца
        maxk=0  # максимум к-го столбца
        for i in range(k, n): # столбцы
            if (maxk<abs(A[i][k])):
                    max_index=i
                    maxk=abs(A[i][k])
        #нашли индекс максимума, теперь меняем строки
        Cb=b[k]
        b[k]=b[max_index]
        b[max_index]=Cb
        
        C=A[k]
        A[k]=A[max_index]
        A[max_index]=C  
        #
        # нормируем строку
        b[k]=b[k]/A[k][k]
        A[k]=[A[k][x]/A[k][k] for x in range(n)] 
        # преобразуем последующие n-k строк
        for m in range(k+1, n):
            b[m]=b[m]-b[k]*A[m][k]
            A[m]=[A[m][x]-A[k][x]*A[m][k] for x in range(n)]            
        #
    #print('A=', np.array(A).round(4))
    #print('b=', np.array(b).round(4))
    
    # обратный ход метода Гаусса
    X=[0]*n # вектор неизвестных
    i=n
    while (i!=0):
        i-=1
        X[i]=b[i]
        for j in range(n-i-1):
            X[i]=X[i]-A[i][n-1-j]*X[i+1]
    return X

In [3]:
A=[[3, 1, 1], [1, 5, 1], [1, 1, 7]]
b=[5, 7, 9]
X1=Gauss(A, b)
print(X1)

[1.0000000000000002, 1.0, 0.9999999999999999]


In [4]:
X= np.linalg.solve(A, b)
print(X)

[1. 1. 1.]


In [5]:
A=[[3, 1, 1], [1, 5, 1], [1, 1, 7]]
b=[5, 7, 9]
print('cond1=', np.linalg.cond(A, 1))

cond1= 4.304347826086957


Как видим, матрица хорошо обусловлена, ответ правильный

# Метод квадратного корня
(только для симметрических матриц)

комментарий к обратному ходу: AX=b <=> (sT*s)*X=b <=> sT*(s*X)=b <=> s*x=b*sT^-1 =:y. y находим из sT*y=b. тогда s*x=y, отсюда ищем х

In [6]:
def sq_root_slae(A, b):
    n= len(b)
    # представим А как A=sT*s, где s - верхняя диагональная матрица
    # вычисляем элементы матрицы s по формулам, расписав покомпонентно A=sT*s
    s=np.zeros((n, n))
    
    # s11^2 = a11
    s[0][0]=A[0][0]**(1/2)
    for i in range(1, n):
        s[0][i]=A[0][i]/s[0][0]
    for i in range(1, n):
        ski=0
        for j in range(i):
            ski+=s[j][i]**2
        s[i][i]=(A[i][i] - ski)**(1/2)    # skk=√(Akk-sum(sik^2, i<k)), k>=2
        for k in range(i, n):
            skj=0
            for j in range(i):
                skj+=s[j][i]*s[j][k]
            s[i][k]=(A[i][k]-skj)/s[i][i] # sij=(Aij-sum(ski*skj, k<i))/sii, j>i
    print('s=', s)
    #print('A=sT*s=',s.transpose().dot(s))
    
    # вычислили матрицу s, см комментарий выше
    # обратный ход
    # вычисляем вектор У. sT*Y=b
    Y=np.zeros(n)
    for i in range(n):
        sk=0
        for k in range(i):
            sk+=s[k][i]*Y[k]
        Y[i]=(b[i]-sk)/s[i][i]
    #print('Y=', Y)
    #
    # вычисляем вектор Х. s*X=Y
    X=np.zeros(n)
    i=n
    while (i!=0):
        i-=1
        sk=0
        for j in range(i+1, n):
            sk+=s[i][j]*X[j]
        X[i]=(Y[i]-sk)/s[i][i]
    return X

In [7]:
A=[[3, 1, 1], [1, 5, 1], [1, 1, 7]]
b=[5, 7, 9]
X2= sq_root_slae(A, b)
print('X2=', X2)
print('X=', X)

s= [[1.73205081 0.57735027 0.57735027]
 [0.         2.1602469  0.3086067 ]
 [0.         0.         2.56347978]]
X2= [1. 1. 1.]
X= [1. 1. 1.]


#  Метод простой итерации

Используется для положительно определенных матриц, однако любая СЛАУ с неособой матрицей может быть приведена к СЛАУ с положительно определенной матрицей, домножением уравнения на АТ слева.
Критерием положительной определённости матрицы может служить положительность её собственных чисел или положительность её главных миноров.

Так же здесь потребовались функции для вычисления нормы матрицы. Я использовала 1-норму

In [8]:
def norm1(A):
    # max суммы столбца
    summ=0
    n=len(A[0])
    # возьмем каждый элемент матрицы по модулю
    for i in range(n):
        A[i]=[abs(x) for x in A[i] ]
    for i in range(n):
        if (summ<np.array(A[i]).sum()):
            summ=np.array(A[i]).sum()
    return summ

In [9]:
def norm1vec(x):
    # возьмем каждый элемент вектора по модулю
    x=[abs(a) for a in x ]
    summ=sum(x)
    return summ

In [10]:
def MPI(A, b, eps):
    eig=np.linalg.eigvals(A) # собственные числа матрицы А
    po=True #положительная определенность матрицы А
    n=len(b)
    for i in eig:
        if i<=0:
            po=False
    #print('po=', po)
    if (po==False):
        A1=np.array(A).transpose().dot(A)
        b=np.array(A).transpose().dot(b)
        A=A1
    m=2/(norm1(A)+eps)
    c=m * np.array(b)
    B=np.eye(n)-m*np.array(A)
    x0=c
    x=[0]*n
    k=0
    while (norm1vec(x-x0)>=eps):
        xk=x # просто запомнили х
        x=c+B.dot(x0)
        x0=xk
        k+=1
    print('кол-во итераций =', k)    
    return(x,k) # возвращаю количество итераций для последующего анализа 

In [11]:
A=[[3, 1, 1], [1, 5, 1], [1, 1, 7]]
b=[5, 7, 9]
X3 = MPI(A, b, 10**(-3))[0]
print('X3=', X3)
X4= MPI(A, b, 10**(-6))[0]
print('X4=', X4)
print('X=', X) # с помощью встроенной функции

кол-во итераций = 47
X3= [1.00015048 1.00023014 1.00048895]
кол-во итераций = 91
X4= [1.00000014 1.00000022 1.00000046]
X= [1. 1. 1.]


#  Результаты работы методов для хорошо обусловленной матрицы

In [12]:
df=pd.DataFrame({'метод': [], 'ответ':[]})
line={'метод': 'Метод Гаусса', 'ответ': np.array(X1).round(5)}
df=df.append(line, ignore_index=True)
line={'метод': 'Метод квадратного корня', 'ответ': np.array(X2).round(5)}
df=df.append(line, ignore_index=True)
line={'метод': 'Метод простой итерации, eps=10^(-3)', 'ответ': np.array(X3).round(4)}
df=df.append(line, ignore_index=True)
line={'метод': 'Метод простой итерации, eps=10^(-6)', 'ответ': np.array(X4).round(7)}
df=df.append(line, ignore_index=True)
line={'метод': 'Встроенная функция python', 'ответ': X.round(7)}
df=df.append(line, ignore_index=True)
display(df)

,метод,ответ
0,Метод Гаусса,"[1.0, 1.0, 1.0]"
1,Метод квадратного корня,"[1.0, 1.0, 1.0]"
2,"Метод простой итерации, eps=10^(-3)","[1.0002, 1.0002, 1.0005]"
3,"Метод простой итерации, eps=10^(-6)","[1.0000001, 1.0000002, 1.0000005]"
4,Встроенная функция python,"[1.0, 1.0, 1.0]"


# Плохо обусловленные матрицы

# Ввод данных

In [13]:
A1=[[1, -1, -1, -1, -1], [0, 1, -1, -1, -1], [0, 0, 1, -1, -1], [0, 0, 0, 1, -1], [0, 0, 0, 0, 1]] + 10**(-3)*np.array([[1, -1, -1, -1, -1], [1, 1, -1, -1, -1], [1, 1, 1, -1, -1], [1, 1, 1, 1, -1], [1, 1, 1, 1, 1]])
b1= [-1, -1, -1, -1, 1]
A2=[[1, -1, -1, -1, -1], [0, 1, -1, -1, -1], [0, 0, 1, -1, -1], [0, 0, 0, 1, -1], [0, 0, 0, 0, 1]] + 10**(-6)*np.array([[1, -1, -1, -1, -1], [1, 1, -1, -1, -1], [1, 1, 1, -1, -1], [1, 1, 1, 1, -1], [1, 1, 1, 1, 1]])
b2= [-1, -1, -1, -1, 1]
A3=[[1, -1, -1, -1, -1, -1], [0, 1, -1, -1, -1, -1], [0, 0, 1, -1, -1, -1], [0, 0, 0, 1, -1, -1], [0, 0, 0, 0, 1, -1], [0, 0, 0, 0, 0, 1]] + 10**(-3)*np.array([[1, -1, -1, -1, -1, -1], [1, 1, -1, -1, -1, -1], [1, 1, 1, -1, -1, -1], [1, 1, 1, 1, -1, -1], [1, 1, 1, 1, 1, -1], [1, 1, 1, 1, 1, -1]])
b3= [-1, -1, -1, -1, -1, 1]
A4=[[1, -1, -1, -1, -1, -1], [0, 1, -1, -1, -1, -1], [0, 0, 1, -1, -1, -1], [0, 0, 0, 1, -1, -1], [0, 0, 0, 0, 1, -1], [0, 0, 0, 0, 0, 1]] + 10**(-6)*np.array([[1, -1, -1, -1, -1, -1], [1, 1, -1, -1, -1, -1], [1, 1, 1, -1, -1, -1], [1, 1, 1, 1, -1, -1], [1, 1, 1, 1, 1, -1], [1, 1, 1, 1, 1, -1]])
b4= [-1, -1, -1, -1, -1, 1]
A5=[[1, -1, -1, -1, -1, -1, -1], [0, 1, -1, -1, -1, -1, -1], [0, 0, 1, -1, -1, -1, -1], [0, 0, 0, 1, -1, -1, -1], [0, 0, 0, 0, 1, -1, -1], [0, 0, 0, 0, 0, 1, -1], [0, 0, 0, 0, 0, 0, 1]] + 10**(-3)*np.array([[1, -1, -1, -1, -1, -1, -1], [1, 1, -1, -1, -1, -1, -1], [1, 1, 1, -1, -1, -1, -1], [1, 1, 1, 1, -1, -1, -1], [1, 1, 1, 1, 1, -1, -1], [1, 1, 1, 1, 1, -1, -1], [1, 1, 1, 1, 1, 1, 1]])
b5= [-1, -1, -1, -1, -1, -1, 1]
A6=[[1, -1, -1, -1, -1, -1, -1], [0, 1, -1, -1, -1, -1, -1], [0, 0, 1, -1, -1, -1, -1], [0, 0, 0, 1, -1, -1, -1], [0, 0, 0, 0, 1, -1, -1], [0, 0, 0, 0, 0, 1, -1], [0, 0, 0, 0, 0, 0, 1]] + 10**(-6)*np.array([[1, -1, -1, -1, -1, -1, -1], [1, 1, -1, -1, -1, -1, -1], [1, 1, 1, -1, -1, -1, -1], [1, 1, 1, 1, -1, -1, -1], [1, 1, 1, 1, 1, -1, -1], [1, 1, 1, 1, 1, -1, -1], [1, 1, 1, 1, 1, 1, 1]])
b6= [-1, -1, -1, -1, -1, -1, 1]
print('A1=', A1)
print('b1=', b1)
print('A2=', A2.round(7))
print('b2=', b2)
print('A3=', A3.round(7))
print('b3=', b3)
print('A4=', A4.round(7))
print('b4=', b4)
print('A5=', A5.round(4))
print('b5=', b5)
print('A6=', A6.round(7))
print('b6=', b6)
cond1=np.linalg.cond(A1, 1)
cond2=np.linalg.cond(A2, 1)
cond3=np.linalg.cond(A3, 1)
cond4=np.linalg.cond(A4, 1)
cond5=np.linalg.cond(A5, 1)
cond6=np.linalg.cond(A6, 1)

A1= [[ 1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.000e-03  1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.000e-03  1.000e-03  1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.000e-03  1.000e-03  1.000e-03  1.001e+00 -1.001e+00]
 [ 1.000e-03  1.000e-03  1.000e-03  1.000e-03  1.001e+00]]
b1= [-1, -1, -1, -1, 1]
A2= [[ 1.000001e+00 -1.000001e+00 -1.000001e+00 -1.000001e+00 -1.000001e+00]
 [ 1.000000e-06  1.000001e+00 -1.000001e+00 -1.000001e+00 -1.000001e+00]
 [ 1.000000e-06  1.000000e-06  1.000001e+00 -1.000001e+00 -1.000001e+00]
 [ 1.000000e-06  1.000000e-06  1.000000e-06  1.000001e+00 -1.000001e+00]
 [ 1.000000e-06  1.000000e-06  1.000000e-06  1.000000e-06  1.000001e+00]]
b2= [-1, -1, -1, -1, 1]
A3= [[ 1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.000e-03  1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.000e-03  1.000e-03  1.001e+00 -1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.000e-03  1.000e-03  1.000e-03  1.001e+00 -1.001e+00 -1.001e+00]
 [ 1.00

## метод Гаусса для плохо обусловленных матриц

In [14]:
# матрица 5х5
X1=Gauss(A1, b1)
print('X1=', X1)
print('cond1=', cond1)
X=np.linalg.solve(A1, b1).round(5)
print('X=', X)
Y1=abs(np.array(X)-np.array(X1))

X2= np.round(Gauss(A2, b2), 7)
print('X2=', X2)
X=np.linalg.solve(A2, b2).round(7)
print('X=', X)
print('cond2=', cond2)
Y2=abs(np.array(X)-np.array(X2))

X1= [-16.87970910776349, -3.9701770271906227, -0.9930387940813349, 0.0, 0.9990009990009993]
cond1= 77.97589669049245
X= [0.    0.    0.    0.    0.999]
X2= [-16.999879  -3.99997   -0.999993   0.         0.999999]
X= [0.       0.       0.       0.       0.999999]
cond2= 79.99792005743845


In [15]:
# матрица 6х6

X3=np.around(Gauss(A3, b3), 5)
print('X3=', X3)
X=np.linalg.solve(A3, b3)
print('X=', X)
print('cond3=', cond3)
Y3=abs(np.array(X)-np.array(X3))


X4=np.around(Gauss(A4, b4),7)
print('X4=', X4)
X=np.linalg.solve(A4, b4).round(7)
print('X=', X)
print('cond4=', cond4)
Y4=abs(np.array(X)-np.array(X4))


X3= [-8.422516e+01 -1.664523e+01 -3.919880e+00 -9.814700e-01  1.880000e-03
  1.000940e+00]
X= [0.03027545 0.0151226  0.00755375 0.0037731  0.00188467 1.00094233]
cond3= 181.95556527828336
X4= [-8.5998184e+01 -1.6999637e+01 -3.9999180e+00 -9.9998100e-01
  2.0000000e-06  1.0000010e+00]
X= [3.200000e-05 1.600000e-05 8.000000e-06 4.000000e-06 2.000000e-06
 1.000001e+00]
cond4= 191.98937661295676


In [16]:
# матрица 7х7

X5=np.around(Gauss(A5, b5), 5)
print('X5=', X5)
X=np.linalg.solve(A5, b5).round(5)
print('X=', X)
print('cond5=', cond5)
Y5=abs(np.array(X)-np.array(X5))


X6=np.around(Gauss(A6, b6), 7)
print('X6=', X6)
X=np.linalg.solve(A6, b6).round(7)
print('X=', X)
print('cond6=', cond6)
Y6=abs(np.array(X)-np.array(X6))

X5= [-495.57219  -82.42886  -16.31894   -3.85448   -0.96984    0.
    0.999  ]
X= [-0.    -0.    -0.    -0.     0.     0.     0.999]
cond5= 400.43355498068877
X6= [-516.9778717  -85.9963121  -16.999297    -3.99985     -0.999969
    0.           0.999999 ]
X= [0.       0.       0.       0.       0.       0.       0.999999]
cond6= 447.94669443877467


# Результат работы метода Гаусса с плохо обусловленными матрицами

In [17]:
df=pd.DataFrame({'матрица': [], 'размерность': [], 'число обусловленности': [],'решение':[], 'вектор смещений':[] })
line={'матрица': 'A1', 'размерность': '5x5', 'число обусловленности': cond1, 'решение': np.array(X1).round(5), 'вектор смещений': np.array(Y1).round(5)}
df=df.append(line, ignore_index=True)
line={'матрица': 'A2', 'размерность': '5x5', 'число обусловленности': cond2, 'решение': np.array(X2).round(5), 'вектор смещений': np.array(Y2).round(5)}
df=df.append(line, ignore_index=True)
line={'матрица': 'A3', 'размерность': '6x6', 'число обусловленности': cond3, 'решение': np.array(X3).round(5), 'вектор смещений': np.array(Y3).round(5)}
df=df.append(line, ignore_index=True)
line={'матрица': 'A4', 'размерность': '6x6', 'число обусловленности': cond4, 'решение': np.array(X4).round(5), 'вектор смещений': np.array(Y4).round(5)}
df=df.append(line, ignore_index=True)
line={'матрица': 'A5', 'размерность': '7x7', 'число обусловленности': cond5, 'решение': np.array(X5).round(5), 'вектор смещений': np.array(Y5).round(5)}
df=df.append(line, ignore_index=True)
line={'матрица': 'A6', 'размерность': '7x7', 'число обусловленности': cond6, 'решение': np.array(X6).round(5), 'вектор смещений': np.array(Y6).round(5)}
df=df.append(line, ignore_index=True)
display(df)

,матрица,размерность,число обусловленности,решение,вектор смещений
0,A1,5x5,77.975897,"[-16.87971, -3.97018, -0.99304, 0.0, 0.999]","[16.87971, 3.97018, 0.99304, 0.0, 0.0]"
1,A2,5x5,79.997920,"[-16.99988, -3.99997, -0.99999, 0.0, 1.0]","[16.99988, 3.99997, 0.99999, 0.0, 0.0]"
2,A3,6x6,181.955565,"[-84.22516, -16.64523, -3.91988, -0.98147, 0.0...","[84.25544, 16.66035, 3.92743, 0.98524, 0.0, 0.0]"
3,A4,6x6,191.989377,"[-85.99818, -16.99964, -3.99992, -0.99998, 0.0...","[85.99822, 16.99965, 3.99993, 0.99998, 0.0, 0.0]"
4,A5,7x7,400.433555,"[-495.57219, -82.42886, -16.31894, -3.85448, -...","[495.57219, 82.42886, 16.31894, 3.85448, 0.969..."
5,A6,7x7,447.946694,"[-516.97787, -85.99631, -16.9993, -3.99985, -0...","[516.97787, 85.99631, 16.9993, 3.99985, 0.9999..."


## метод простой итерации для плохо обусловленных матриц


In [18]:
# матрица 5х5
eps=10**(-3)
X1, k1=MPI(A1, b1, eps)
print('X1=', X1)
cond1=np.linalg.cond(A1, 1)
print('cond1=', cond1)
X=np.linalg.solve(A1, b1).round(5)
print('X=', X)
Y1=abs(np.array(X)-np.array(X1))


eps=10**(-6)
X2, k2=MPI(A2, b2, eps)
print('X2=', X2)
X=np.linalg.solve(A2, b2).round(7)
print('X=', X)
print('cond2=', cond2)
Y2=abs(np.array(X)-np.array(X2))

кол-во итераций = 56
X1= [-1.74690526e-03  4.06145399e-04 -2.75722692e-02 -1.97036475e+00
  9.99000383e-01]
cond1= 9.956419319672138
X= [-0.00000e+00 -1.60000e-04 -2.74600e-02 -1.97038e+00  9.99000e-01]
кол-во итераций = 92
X2= [-1.70302657e-06  2.78428440e-07 -2.80300531e-05 -1.99997000e+00
  9.99999000e-01]
X= [-0.00000e+00 -0.00000e+00 -2.80000e-05 -1.99997e+00  9.99999e-01]
cond2= 79.99792005743845


In [19]:
# матрица 6х6
eps=10**(-3)
X3, k3=MPI(A3, b3, eps)
print('X3=', X3)
X=np.linalg.solve(A3, b3)
print('X=', X)
print('cond3=', cond3)
Y3=abs(np.array(X)-np.array(X3))

eps=10**(-6)
X4, k4=MPI(A4, b4, eps)
print('X4=', X4)
X=np.linalg.solve(A4, b4).round(7)
print('X=', X)
print('cond4=', cond4)
Y4=abs(np.array(X)-np.array(X4))

кол-во итераций = 70
X3= [ 2.45601392e-03 -1.49743071e-03 -2.82418929e-04 -5.76811747e-02
 -1.94155122e+00  1.00094164e+00]
X= [-9.61528364e-09 -4.80764192e-06 -8.02341306e-04 -5.75719046e-02
 -1.94156427e+00  1.00094233e+00]
cond3= 181.95556527828336
кол-во итераций = 122
X4= [ 2.20399422e-06 -5.02036259e-07  8.01719899e-08 -6.00063439e-05
 -1.99994000e+00  1.00000100e+00]
X= [-0.000000e+00 -0.000000e+00 -0.000000e+00 -6.000000e-05 -1.999940e+00
  1.000001e+00]
cond4= 191.98937661295676


In [20]:
eps=10**(-3)
X5, k5=MPI(A5, b5, eps)
print('X5=', X5)
X=np.linalg.solve(A5, b5).round(5)
print('X=', X)
print('cond4=', cond5)
Y5=abs(np.array(X)-np.array(X5))

eps=10**(-6)
X6, k6=MPI(A6, b6, eps)
print('X6=', X6)
X=np.linalg.solve(A6, b6).round(7)
print('X=', X)
print('cond6=', cond6)
Y6=abs(np.array(X)-np.array(X6))

кол-во итераций = 82
X5= [-1.15756125e-03  2.32099783e-03 -1.62758342e-03 -2.66084503e-03
 -1.10165139e-01 -1.88464781e+00  9.98999977e-01]
X= [-0.00000e+00 -0.00000e+00 -5.00000e-05 -3.26000e-03 -1.10030e-01
 -1.88467e+00  9.99000e-01]
cond4= 400.43355498068877
кол-во итераций = 152
X6= [-3.14512509e-06  9.74868936e-07 -2.18837092e-07  3.14280903e-08
 -1.19993165e-04 -1.99987801e+00  9.99999000e-01]
X= [-0.000000e+00  0.000000e+00 -0.000000e+00 -0.000000e+00 -1.200000e-04
 -1.999878e+00  9.999990e-01]
cond6= 447.94669443877467


# Результат работы метода простой итерации с плохо обусловленными матрицами

In [21]:
df=pd.DataFrame({'матрица': [], 'размерность': [], 'число обусл': [], 'решение': [], 'вектор смещения': [], 'итерации':[] })
line={'матрица': 'A1', 'размерность': '5x5', 'число обусл': cond1, 'решение': np.array(X1).round(5), 'вектор смещения': np.array(Y1).round(4), 'итерации': k1}
df=df.append(line, ignore_index=True)
line={'матрица': 'A2', 'размерность': '5x5', 'число обусл': cond2, 'решение': np.array(X2).round(7), 'вектор смещения': np.array(Y2).round(7), 'итерации': k2}
df=df.append(line, ignore_index=True)
line={'матрица': 'A3', 'размерность': '6x6', 'число обусл': cond3, 'решение': np.array(X3).round(5), 'вектор смещения': np.array(Y3).round(4), 'итерации': k3}
df=df.append(line, ignore_index=True)
line={'матрица': 'A4', 'размерность': '6x6', 'число обусл': cond4, 'решение': np.array(X4).round(7), 'вектор смещения': np.array(Y4).round(7), 'итерации': k4}
df=df.append(line, ignore_index=True)
line={'матрица': 'A5', 'размерность': '7x7', 'число обусл': cond5, 'решение': np.array(X5).round(5), 'вектор смещения': np.array(Y5).round(4), 'итерации': k5}
df=df.append(line, ignore_index=True)
line={'матрица': 'A6', 'размерность': '7x7', 'число обусл': cond6, 'решение': np.array(X6).round(7), 'вектор смещения': np.array(Y6).round(7), 'итерации': k6}
df=df.append(line, ignore_index=True)
display(df)

,матрица,размерность,число обусл,решение,вектор смещения,итерации
0,A1,5x5,9.956419,"[-0.00175, 0.00041, -0.02757, -1.97036, 0.999]","[0.0017, 0.0006, 0.0001, 0.0, 0.0]",56.0
1,A2,5x5,79.997920,"[-1.7e-06, 3e-07, -2.8e-05, -1.99997, 0.999999]","[1.7e-06, 3e-07, 0.0, 0.0, 0.0]",92.0
2,A3,6x6,181.955565,"[0.00246, -0.0015, -0.00028, -0.05768, -1.9415...","[0.0025, 0.0015, 0.0005, 0.0001, 0.0, 0.0]",70.0
3,A4,6x6,191.989377,"[2.2e-06, -5e-07, 1e-07, -6e-05, -1.99994, 1.0...","[2.2e-06, 5e-07, 1e-07, 0.0, 0.0, 0.0]",122.0
4,A5,7x7,400.433555,"[-0.00116, 0.00232, -0.00163, -0.00266, -0.110...","[0.0012, 0.0023, 0.0016, 0.0006, 0.0001, 0.0, ...",82.0
5,A6,7x7,447.946694,"[-3.1e-06, 1e-06, -2e-07, 0.0, -0.00012, -1.99...","[3.1e-06, 1e-06, 2e-07, 0.0, 0.0, 0.0, 0.0]",152.0


## Метод квадратного корня для плохо обусловленных систем

Плохо обусловленные матрицы в примере не являются симметрическими, метод квадратного корня к ним не применить, убедимся в этом

In [22]:
# матрица 5х5
eps=10**(-3)
A=[[1, -1, -1, -1, -1], [0, 1, -1, -1, -1], [0, 0, 1, -1, -1], [0, 0, 0, 1, -1], [0, 0, 0, 0, 1]] + eps*np.array([[1, -1, -1, -1, -1], [1, 1, -1, -1, -1], [1, 1, 1, -1, -1], [1, 1, 1, 1, -1], [1, 1, 1, 1, 1]])
b= [-1, -1, -1, -1, 1]
X1=np.round(sq_root_slae(A, b), 5)
print('X1=', X1)
cond7=np.linalg.cond(A, 1)
print('cond7=', cond1)
X=np.linalg.solve(A, b).round(5)
print('X=', X)
Y1=abs(np.array(X)-np.array(X1))

eps=10**(-6)
A=[[1, -1, -1, -1, -1], [0, 1, -1, -1, -1], [0, 0, 1, -1, -1], [0, 0, 0, 1, -1], [0, 0, 0, 0, 1]] + 10**(-6)*np.array([[1, -1, -1, -1, -1], [1, 1, -1, -1, -1], [1, 1, 1, -1, -1], [1, 1, 1, 1, -1], [1, 1, 1, 1, 1]])
b= [-1, -1, -1, -1, 1]
X2= np.round(sq_root_slae(A, b), 7)
print('X2=', X2)
X=np.linalg.solve(A, b).round(7)
print('X=', X)
cond8=np.linalg.cond(A, 1)
print('cond8=', cond2)
Y2=abs(np.array(X)-np.array(X2))

s= [[ 1.00049988 -1.00049988 -1.00049988 -1.00049988 -1.00049988]
 [ 0.                 nan         nan         nan         nan]
 [ 0.          0.                 nan         nan         nan]
 [ 0.          0.          0.                 nan         nan]
 [ 0.          0.          0.          0.                 nan]]
X1= [nan nan nan nan nan]
cond7= 9.956419319672138
X= [0.    0.    0.    0.    0.999]
s= [[ 1.00000050e+00 -1.00000050e+00 -1.00000050e+00 -1.00000050e+00
  -1.00000050e+00]
 [ 0.00000000e+00  1.49011612e-08 -1.34217862e+08 -1.34217862e+08
  -1.34217862e+08]
 [ 0.00000000e+00  0.00000000e+00             nan             nan
              nan]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00             nan
              nan]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
              nan]]
X2= [nan nan nan nan nan]
X= [0.       0.       0.       0.       0.999999]
cond8= 79.99792005743845


<ipython-input-6-f3c4ad2feede>:20: RuntimeWarning: invalid value encountered in double_scalars
  s[i][k]=(A[i][k]-skj)/s[i][i] # sij=(Aij-sum(ski*skj, k<i))/sii, j>i
<ipython-input-6-f3c4ad2feede>:15: RuntimeWarning: invalid value encountered in double_scalars
  s[i][i]=(A[i][i] - ski)**(1/2)    # skk=√(Akk-sum(sik^2, i<k)), k>=2


s= [[ 1.  1.  1.  1.  1.]
 [ 0. nan nan nan nan]
 [ 0.  0. nan nan nan]
 [ 0.  0.  0. nan nan]
 [ 0.  0.  0.  0. nan]]


<ipython-input-6-f3c4ad2feede>:20: RuntimeWarning: invalid value encountered in double_scalars
  s[i][k]=(A[i][k]-skj)/s[i][i] # sij=(Aij-sum(ski*skj, k<i))/sii, j>i
